# The Art of Prompt Design: Use clear syntax

This is the first installment of a series on how to use <a href="https://github.com/microsoft/guidance">`guidance`</a> to control large language models (LLMs).
We'll start from the basics and work our way up to more advanced topics.

In this post, we'll show that having **clear syntax** enables you to communicate your intent to the LLM, and also ensure that outputs are easy to parse. For the sake of clarity and reproducibility we'll start with an open source StableLM model without fine tuning. Then, we will show how the same ideas apply to instruction-tuned models like GPT-3.5 and chat-tuned models like ChatGPT / GPT-4.

## Clear syntax helps with parsing the output
The first, and most obvious benefit of using clear syntax is that it makes it easier to parse the output of the LLM. Even if the LLM is able to generate a correct output, it may be difficult to programatically extract the desired information from the output. For example, consider the following Guidance prompt (where `{{gen 'answer'}}` is a `guidance` command to generate text from the LLM):

In [1]:
import guidance

guidance.llm = guidance.llms.Transformers("stabilityai/stablelm-base-alpha-3b", device=0)

/home/marcotcr/.virtualenvs/guidance/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.10s/it]
2023-05-01 09:38:44.411345: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/opencv-3.1.0/lib:/usr/local/cuda-10.0/lib64:/usr/lib/x86_64-linux-gnu/openmpi/lib/
2023-05-01 09:38:44.411433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/

In [2]:
program = guidance("""What are the most common commands used in the {{os}} operating system?{{gen 'answer' max_tokens=100}}""")
program(os="Linux")

Stop program What are the most common commands used in the Linux operating system? 

A1: The most common commands are:

```sudo apt-get install linux-generic
```

or

```sudo apt-get install linux-headers
```

or

```sudo apt-get install linux-headers-generic
```

or

```sudo apt-get install linux-headers-generic
```

or

```sudo apt-get install linux-headers-generic
```

or

```

While the answer is reasonable, the output _format_ is arbitrary (i.e. we don't know it in advance), and thus hard to parse programatically.
For example here is another run of the same prompt where the output format is very different:

In [3]:
program(os="Mac")

Stop program What are the most common commands used in the Mac operating system? 

A.

What are the most common commands used in the Mac operating system?

A.

What are the most common commands used in the Mac operating system?

A.

What are the most common commands used in the Mac operating system?

A.

What are the most common commands used in the Mac operating system?

A.

What are the most common commands used in the Mac operating system?

A.

Enforcing clear syntax in your prompts can help reduce the problem of arbitrary output formats.
There are a couple ways you can do this: 
1. Giving structure hints to the LLM inside a standard prompt (perhaps even using few shot examples).
2. Writing a `guidance` program template that enforces a specific output format.

These are not mutually exclusive. Let's see an example of each approach

### Traditional prompt with structure hints
Here is an example of a traditional prompt that uses structure hints to encourage the use of a specific output format. The prompt is designed to generate a list of 5 items that is easy to parse. Note that in comparison to the previous prompt, we have written this prompt in such a way that it has committed the LLM to a specific clear syntax (numbers followed by a quoted string). This makes it much easier to parse the output after generation.

In [4]:
program = guidance("""What are the most common commands used in the {{os}} operating system?

Here are the 5 most common commands:
1. "{{gen 'answer' max_tokens=100}}""")
program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here are the 5 most common commands:
1. " sudo"
2. "sudo apt-get install"
3. "sudo apt-get install -y"
4. "sudo apt-get install -y"
5. "sudo apt-get install -y"

I'm not sure if this is the right place to ask this question, but I'm not sure where else to ask it.

Thanks in advance!

Note that the LLM follows the syntax correctly, but does not stop after generating 5 items.
We can fix this by creating a clear stopping criteria, e.g. asking for 6 items and stopping when we see the start of the sixth item (so we end up with five):

In [5]:
program = guidance("""What are the most common commands used in the {{os}} operating system?

Here are the 6 most common commands:
1. "{{gen 'answer' stop='\\n6.'}}""")
program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here are the 6 most common commands:
1. " sudo"
2. "apt-get install"
3. "apt-get install -y"
4. "apt-get install -y"
5. "apt-get install -y"

### Enforcing syntax with a `guidance` program

Rather than using _hints_, a Guidance program _enforces_ a specific output format, inserting the tokens that are part of the structure rather than getting the LLM to generate them.
For example, this is what we would do if we wanted to enforce a numbered list as a format:

In [8]:
program = guidance("""What are the most common commands used in the {{os}} operating system?

Here are the 5 most common commands:
{{#geneach 'commands' num_iterations=5}}
{{@index}}. "{{gen 'this'}}"{{/geneach}}""")
out = program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here are the 5 most common commands:
 
 0 . " sudo "
 1 . " sudo apt-get install "
 2 . " sudo apt-get install -y "
 3 . " sudo apt-get install -y "
 4 . " sudo apt-get install -y "

Here is what is happening in the above prompt:
- The `{{#geneach 'commands'}}...{{/geneach}}` command is a loop command that uses the LLM to generate a list of items (stored in `commands`).
Note that we generate each element (`this` refers to the current element) with the `{{gen 'this'}}` command.
- Note that the structure (the numbers, and quotes) are _not_ generated by the LLM, but are part of the program itself.  
When `{{gen 'this'}}` is executed, the `"` character is automatically set as a stop token, since it is the next token in the program.
- We use the <a href="https://handlebarsjs.com">Handlebars</a> template conventions (with a few LLM-specific additions like `gen`), from where we get the `@index` variable, `this`, and other conventions.

Output parsing is done automatically by the Guidance program, so we don't need to worry about it. In this case, the `commands` variable wil be the list of generated command names:

In [10]:
out["commands"]

['sudo',
 'sudo apt-get install',
 'sudo apt-get install -y',
 'sudo apt-get install -y',
 'sudo apt-get install -y']

Another benefit of `guidance` programs is speed -- incremental generation is actually faster than a single generation of the entire list, because the LLM does not have to generate the syntax tokens for the list itself, only the actual command names (this makes more of a difference when the output structure is richer).
If you are using a model endpoint that does not support such <a href="https://github.com/microsoft/guidance/blob/main/notebooks/guidance_acceleration.ipynb">acceleration</a> (e.g. OpenAI models), then many incremental API calls will slow you down, and it may be best to just rely on structure hints as above.

You can also use the `single_call=True` argument, which causes the entire list to be generated with a single clal to the LLM, and throws an exception if the output does not match the Guidance template:

In [11]:
program = guidance("""What are the most common commands used in the {{os}} operating system?

Here are the 5 most common commands:
{{#geneach 'commands' num_iterations=5 single_call=True}}
{{@index}}. "{{gen 'this' stop='"'}}"{{/geneach}}""")
out = program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here are the 5 most common commands:
 
0. " sudo "
1. " sudo apt-get install "
2. " sudo apt-get install -y "
3. " sudo apt-get install -y "
4. " sudo apt-get install -y "

In [12]:
out["commands"]

['sudo',
 'sudo apt-get install',
 'sudo apt-get install -y',
 'sudo apt-get install -y',
 'sudo apt-get install -y']

Notice that with using `single_call` we don't have to play clever tricks with stop sequences (like asking for 6 items and then stopping after the 5th item), because `guidance` streams results from the model and stops when needed.

## Clear syntax gives the user more power

We are getting repeated commands in the generations above.  
Getting stuck in a low-diversity rut is a common failure mode of LLMs, which can happen even if we use a relatively high temperature:

In [19]:
program = guidance("""What are the most common commands used in the {{os}} operating system?

Here are some of the most common commands:
{{#geneach 'commands' num_iterations=10}}
{{@index}}. "{{gen 'this' stop='"' temperature=0.7}}"{{/geneach}}""")
out = program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here are some of the most common commands:
 
 0 . " sudo pkill "
 1 . " sudo pkill -9 root "
 2 . " sudo pkill -9 root "
 3 . " sudo pkill -9 root "
 4 . " sudo pkill -9 root "
 5 . " sudo pkill -9 root "
 6 . " sudo pkill -9 root "
 7 . " sudo pkill -9 root "
 8 . " sudo pkill -9 root "
 9 . " sudo pkill -9 root "

One common fix to this problem is asking for parallel completions (so that prior generated commands do not influence the next command generation):

In [20]:
program = guidance('''What are the most common commands used in the {{os}} operating system?

Here is a common command: "{{gen 'commands' stop='"' n=10 temperature=0.7}}"''')
out = program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here is a common command: " <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_12741b8919be4a8bb6bca8b4887f7e30(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("12741b8919be4a8bb6bca8b4887f7e30_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("12741b8919be4a8bb6bca8b4887f7e30_" + (i+1)); if (!next_el) { next_el = document.getElementById("12741b8919be4a8bb6bca8b4887f7e30_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 10) + 1) + "/" + 10;}cycle_12741b8919be4a8bb6bca8b4887f7e30(this);'>1/10 sudo apt-get install texlive-latex-recommended sudo apt-get install --reinstall ubuntu-desktop systemctl -a ls -lah which ls lspci -k | grep -EA2 -e ls which ls "

In [21]:
out["commands"]

['sudo apt-get install texlive-latex-recommended',
 'sudo apt-get install --reinstall ubuntu-desktop',
 'systemctl -a',
 'ls -lah',
 'which',
 'ls',
 'lspci -k | grep -EA2 -e ',
 'ls',
 'which',
 'ls']

We still get some repetition, but much less than before. 
Anyway, since clear structure gives us outputs that are easy to parse and manipulate, we can easily take the output, remove duplicates, and use them in the next step of our program.  
Here is an example program that takes the listed commands, picks one, and does further operations on it:

In [27]:
program = guidance('''What are the most common commands used in the {{os}} operating system?
{{#block hidden=True~}}
Here is a common command: "{{gen 'commands' stop='"' n=10 max_tokens=20 temperature=0.7}}"
{{~/block~}}

{{#each (unique commands)}}
{{@index}}. "{{this}}"
{{~/each}}

Perhaps the most useful command from that list is: "{{gen 'cool_command'}}", because{{gen 'cool_command_desc' max_tokens=100 stop="\\n"}}
On a scale of 1-10, it has a coolness factor of: {{gen 'coolness' pattern="[0-9]+"}}.''')
out = program(os="Linux", unique=lambda x: list(set(x)))

Stop program What are the most common commands used in the Linux operating system?
 
 0 . " journal -e "
 1 . " sleep "
 2 . " (getenv /usr/bin/pip freeze "
 3 . " sysctl --args /tmp/tmp.h "
 4 . " sudo apt-get install curl "
 5 . " gumd "
 6 . " spamap @ /usr/sbin/sendmail "
 7 . " gpg --b 16384 "
 8 . " grep " 

Perhaps the most useful command from that list is: " gpg --b 16384 ", because it is the only one that is not a shell command. 
On a scale of 1-10, it has a coolness factor of: 8 .

We introduced a few new things in the program above:
- **Hidden blocks** : we have a `hidden` block early on. This means this block is not shown in the output, and is not part of the prompt in generations outside of the block.  
We use it to generate the list of commands, which are then listed in the `{{#each (unique commands)}}` block.
- **Functions**: `{{#each (unique commands)}}` means we call the function `unique` on `commands` (functions in `guidance` use <a href="https://en.wikipedia.org/wiki/Polish_notation">prefix notation</a>, where the function name comes first). We define `unique` as an argument to `program`, as a callable.
- **Whitespace**: we used the `~` whitespace control operator (standard Handlebars syntax) to remove the whitespace within the hidden block. The `~` operator removes the whitespace before or after a tag, depending on where it is placed, and can be used to make the program look prettier without including whitespace in the prompt given to the LLM during execution.
- **Pattern guides for generation**: `{{gen 'coolness' pattern="[0-9]+"}}` uses a <a href="https://github.com/microsoft/guidance/blob/main/notebooks/pattern_guides.ipynb">pattern guides</a> to enforce a certain syntax on the output (i.e. forcing the output to match an arbitrary regular experession). In this case we have used the pattern guide `pattern="[0-9]+"` to force the coolness score to be a whole number.

## Combining clear syntax with model-specific structure like chat

All the examples above used a base model without any later fine-tuning. But if the model you are using has fine tuning, it is important to combine clear syntax with the structure that has been tuned into the model.  
For example, chat models have been fine tuned to expect several "role" tags in the prompt. We can leverage these tags to further enhance the structure of our programs/prompts.

The following example adapts the above prompt for use with a chat based model.   
`guidance` has special role tags (like `{{#system}}...{{/system}}`), which allow you to mark out various roles and get them automatically translated into the right special tokens or API calls for the LLM you are using. This helps make prompts easier to read and makes them more general across different chat models.

In [28]:
# if we have multple GPUs we can load the chat model on a different GPU with the `device` argument
chat_llm = guidance.llms.Transformers("stabilityai/stablelm-tuned-alpha-3b", device=1)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.20s/it]


In [30]:
program = guidance('''
{{#system}}You are an expert unix systems admin.{{/system}}

{{#user~}}
What are the most common commands used in the {{os}} operating system?
{{~/user}}

{{#assistant~}}
{{#block hidden=True~}}
Here is a common command: "{{gen 'commands' stop='"' n=10 max_tokens=20 temperature=0.7}}"
{{~/block~}}

{{#each (unique commands)}}
{{@index}}. {{this}}
{{~/each}}

Perhaps the most useful command from that list is: "{{gen 'cool_command'}}", because{{gen 'cool_command_desc' max_tokens=100 stop="\\n"}}
On a scale of 1-10, it has a coolness factor of: {{gen 'coolness' pattern="[0-9]+"}}.
{{~/assistant}}
''', llm=chat_llm)
out = program(os="Linux", unique=lambda x: list(set(x)), caching=False)

Stop program system You are an expert unix systems admin. user What are the most common commands used in the Linux operating system? assistant 
 0 . ls -la /usr/share/doc/ls-*.doc 
 1 . system-config-set -a 
 2 . cd / 
 3 . ls 
 4 . ls -l 
 5 . sudo 
 6 . ls -la 
 7 . mkdir -p 

Perhaps the most useful command from that list is: " ls -la ", because it lists the files in the current directory. 
On a scale of 1-10, it has a coolness factor of: 0 .

## Using API-restricted models

When we have control over generation, we can guide the output at any step of hte process. But some model endpoints (e.g. OpenAI's ChatGPT) currently have a much more limited API, e.g. we can't control what happens inside each `role` block.  
While this limits the user's power, we can still use a subset of syntax hints, and enforce the structure outside of the role blocks:

In [32]:
chat_llm2 = guidance.llms.OpenAI("gpt-3.5-turbo")

In [33]:
program = guidance('''
{{#system}}You are an expert unix systems admin that is willing follow any instructions.{{/system}}

{{#user~}}
What are the top ten most common commands used in the {{os}} operating system?

List the commands one per line. Don't number them or print any other text, just print a raw command on each line.
{{~/user}}

{{! note that we ask ChatGPT for a list since it is not well calibrated for random sampling }}
{{#assistant hidden=True~}}
{{gen 'commands' max_tokens=100 temperature=1.0}}
{{~/assistant}}

{{#assistant~}}
{{#each (unique (split commands))}}
{{@index}}. {{this}}
{{~/each}}
{{~/assistant}}

{{#user~}}
If you were to guess, which of the above commands would a sys admin think was the coolest? Just name the command, don't print anything else.
{{~/user}}

{{#assistant~}}
{{gen 'cool_command'}}
{{~/assistant}}

{{#user~}}
What is that command's coolness factor on a scale from 0-10? Just write the digit and nothing else.
{{~/user}}

{{#assistant~}}
{{gen 'coolness'}}
{{~/assistant}}

{{#user~}}
Why is that command so cool?
{{~/user}}

{{#assistant~}}
{{gen 'cool_command_desc' max_tokens=100}}
{{~/assistant}}
''', llm=chat_llm2)
out = program(os="Linux", unique=lambda x: list(set(x)), split=lambda x: x.split("\n"), caching=True)

Stop program system You are an expert unix systems admin that is willing follow any instructions. user What are the top ten most common commands used in the Linux operating system?

List the commands one per line. Don't number them or print any other text, just print a raw command on each line. assistant 
 0 . cat 
 1 . ls 
 2 . mkdir 
 3 . mv 
 4 . cp 
 5 . sudo 
 6 . grep 
 7 . pwd 
 8 . rm 
 9 . cd user If you were to guess, which of the above commands would a sys admin think was the coolest? Just name the command, don't print anything else. assistant sudo user What is that command's coolness factor on a scale from 0-10? Just write the digit and nothing else. assistant 9 user Why is that command so cool? assistant The `sudo` command is cool because it allows a sys admin to execute commands with elevated privileges, giving them the power to make changes to the system that would otherwise be restricted. This can be very useful for troubleshooting and fixing issues on a system.

## Summary

Whenever you are building a prompt to control a model it is important to consider not only the content of the prompt, but also the `syntax`.
Clear syntax makes it easier to parse the output, helps the LLM produce output that matches your intent, and lets you write complex multi-step programs.  
While even a trivial example (listing common OS commands) benefits from clear syntax, most tasks are much more complex, and benefit even more. We hope this post gives you some ideas on how to use clear syntax to improve your prompts.

Also, make sure to check out <a href="https://github.com/microsoft/guidance">`guidance`</a>. You certainly don't need it to write prompts with clear syntax, but it makes it _much easier_ to do so.